In [1]:
import numpy as np
import pandas as pd
####### import data into neo4j ##### 
from neo4j import GraphDatabase
from py2neo import Graph

In [2]:
#data = pd. read_csv("C:/Users/Iro Sfoungari/Desktop/THESIS DATA/Trial Graph/Tria_Topic_Extracted.csv")#Sort out blank abstracts
topics = pd. read_csv("C:/Users/Iro Sfoungari/Desktop/THESIS DATA/FINAL_DATABASE/Topics/0_Topic.csv")#Sort out blank abstracts


In [3]:
topics.head(5)

,cord_uid,Topic,Probability
0,ug7v899j,"0.055*virus,0.017*human,0.017*viral,0.014*gene...",0.029789
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573
2,ejv2xln0,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.056605
3,2b73a28n,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.112175
4,9785vg6d,"0.099*cell,0.024*infection,0.023*expression,0....",0.283934


In [5]:
#topics["Topic"]=topics["Topic"].str.replace(", ", ",")

In [6]:
len(topics)

269233

In [7]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="1111")

In [8]:
k=graph.run("MATCH (e:Entity)-[IDENTIFIED_IN]->(p:Publication) RETURN e.name, p.id").data()


In [9]:
df = pd.DataFrame(k)

In [10]:
len(df)

3121152

In [11]:
data2=df.groupby('p.id', sort=False).agg(', '.join).reset_index()


In [12]:
data2.head(10)

,p.id,e.name
0,q5ojwewk,"coronavirus family, CT, enveloped positive s..."
1,33bi0rcw,droplets
2,kbhv9ukg,"microbicidal proteins, citrullinated histone..."
3,3xwuq5h0,coronavirus disease 2019
4,xtkbe2df,"COVID-19, participants"
5,jnih7l5q,"coronavirus, acute respiratory syndrome coron..."
6,g60ot51i,"insulin signalling-related molecules, transf..."
7,5w4z8y94,"zone, water"
8,l40uugwv,"COVID-19 SignSym, https://clamp uth edu/covi..."
9,eyre2wvz,"https://clamp uth edu/covid/nlp, COVID-19, C..."


In [13]:
entities=data2
#entities.head(5)

In [14]:
entities.rename(columns = {'e.name':'Entities'}, inplace = True)

In [15]:
entities.rename(columns = {'p.id':'id'}, inplace = True)

In [16]:
entities.head(5)

,id,Entities
0,q5ojwewk,"coronavirus family, CT, enveloped positive s..."
1,33bi0rcw,droplets
2,kbhv9ukg,"microbicidal proteins, citrullinated histone..."
3,3xwuq5h0,coronavirus disease 2019
4,xtkbe2df,"COVID-19, participants"


In [17]:
topics['Entities'] = topics.cord_uid.map(entities.set_index('id')['Entities'])


In [24]:
topics.head(5)

,cord_uid,Topic,Probability,Entities
0,ug7v899j,"0.055*virus,0.017*human,0.017*viral,0.014*gene...",0.029789,"Patients,M. pneumoniae cultures,Jeddah,Mycopl..."
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573,phagocyte-derived heme proteins myeloperoxida...
2,ejv2xln0,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.056605,"metalloproteinase,collectin,SP-D,complex orga..."
3,2b73a28n,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.112175,"ET-1,Endothelin-1,smooth muscle,airways,pulmo..."
4,9785vg6d,"0.099*cell,0.024*infection,0.023*expression,0....",0.283934,"adhesion molecules,chemoattractant cytokines,..."


In [20]:
#topics["Topic"]=topics["Topic"].astype(str)

In [21]:
topics["Entities"]=topics["Entities"].astype(str)

In [22]:
topics["Entities"]=topics["Entities"].str.replace(", ", ",")

In [23]:
topics["Entities"]=topics["Entities"].str.replace(", ", ",")

In [25]:
#topics["disease"]=topics["disease"].str.replace("  ", " ")


In [26]:
from re import search
temp_list = []


class Solution:
    def solve(self, s0, s1):
        temp_list = []
        k=s0
        s0 = ''.join([i for i in s0 if not i.isdigit()])
        s0 = s0.replace(".*", "")
        l1 = s0.split(",")
        l2= s1.split(",")
        l3=k.split(",")
        x= list(set(l1).intersection(set(l2)))
        for idx, word in enumerate(x):
            for i in range(len(l3)):
                g = ''.join([i for i in l3[i] if not i.isdigit()])
                g= g.replace(".*", "")
                if word==g:
                    temp_list.append(l3[i])
        k= ",".join(str(x) for x in temp_list)
        return k

ob = Solution()
S = '0.053*protein,0.023*virus,0.022*sars,0.020*viral,0.018*bind,0.013*target,0.012*host,0.010*drug,0.010*structure,0.010*rna'
T=' cellular factors,hnRNP A1,250 kDa cellular protein,mutant protein,wild-type hnRNP A1,nuclear transport domain,nucleocapsid protein,hnRNP A1 mutant,nuclear ribonucleoprotein,dominant-negative hnRNP A1 mutant,MHV polymerase gene product,cellular proteins,mouse,virus,viral,viral RNA,viral mRNA,defective-interfering RNAs,viral RNA,MHV RNA,mouse hepatitis virus,MHV,hepatitis virus,protein,nuclear,nucleus,cellular'
print(ob.solve(S,T))

#print(ob.solve(k,a))
#ob.solve(S,T)

0.020*viral,0.023*virus,0.053*protein


In [27]:
ob = Solution()
topics['Intersection'] = topics.apply(lambda x: ob.solve(x.Topic, x.Entities), axis=1)

In [31]:
topics.head(80)

,cord_uid,Topic,Probability,Entities,Intersection
0,ug7v899j,"0.055*virus,0.017*human,0.017*viral,0.014*gene...",0.029789,"Patients,M. pneumoniae cultures,Jeddah,Mycopl...",
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573,phagocyte-derived heme proteins myeloperoxida...,0.014*lung
2,ejv2xln0,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.056605,"metalloproteinase,collectin,SP-D,complex orga...",
3,2b73a28n,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.112175,"ET-1,Endothelin-1,smooth muscle,airways,pulmo...",
4,9785vg6d,"0.099*cell,0.024*infection,0.023*expression,0....",0.283934,"adhesion molecules,chemoattractant cytokines,...",
...,...,...,...,...,...
75,1y5nej0m,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.078632,"neutralizing human monoclonal antibody KZ52,n...","0.020*viral,0.023*virus"
76,fvfjz7al,"0.035*study,0.019*datum,0.017*analysis,0.015*r...",0.249692,"Nile virus,Canada,Alberta",
77,lntn11a9,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.019907,granulocyte-macrophage colony-stimulating fac...,
78,nzk8m912,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.046988,"brain proteins,patients,patient,brain,cognitiv...",


In [397]:
#csv_data = topics.to_csv(r'C:/Users/Iro Sfoungari/Desktop/ayeayeaye2.csv', index = False)

In [32]:
class Solution1:
    def solve1(self, s0, s1):
        s0List = s0.split(",")
        x=s0List
        s1List = s1.split(",")
        y=s1List
        x= [i for i in y if i not in x]
        k= ", ".join(str(j) for j in x)

        return k
            
    
ob = Solution1()
S = '0.055*respiratory specimens,0.017*upper respiratory tract infections,0.017*viral,0.014*gene,0.013*sequence,0.013*genome,0.013*rna,0.012*strain,0.010*sample,0.010*detect'
T='0.017*upper respiratory tract infections,0.055*respiratory specimens'
print(ob.solve1(T,S))

0.017*viral, 0.014*gene, 0.013*sequence, 0.013*genome, 0.013*rna, 0.012*strain, 0.010*sample, 0.010*detect


In [33]:
topics["Intersection"]=topics["Intersection"].astype(str)

In [34]:
ob = Solution1()
topics['Final_Topic_Keywords'] = topics.apply(lambda x: ob.solve1(x.Intersection, x.Topic), axis=1)

In [35]:
topics.head(50)

,cord_uid,Topic,Probability,Entities,Intersection,Final_Topic_Keywords
0,ug7v899j,"0.055*virus,0.017*human,0.017*viral,0.014*gene...",0.029789,"Patients,M. pneumoniae cultures,Jeddah,Mycopl...",,"0.055*virus, 0.017*human, 0.017*viral, 0.014*g..."
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573,phagocyte-derived heme proteins myeloperoxida...,0.014*lung,"0.074*patient, 0.018*treatment, 0.016*case, 0...."
2,ejv2xln0,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.056605,"metalloproteinase,collectin,SP-D,complex orga...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*..."
3,2b73a28n,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.112175,"ET-1,Endothelin-1,smooth muscle,airways,pulmo...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*..."
4,9785vg6d,"0.099*cell,0.024*infection,0.023*expression,0....",0.283934,"adhesion molecules,chemoattractant cytokines,...",,"0.099*cell, 0.024*infection, 0.023*expression,..."
5,zjufx4fo,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.347580,"subgenomic RNA molecules,TRS mutants,equine,Ni...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*..."
6,5yhe786e,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.303051,"patient,patients,organ,organ failure,haemoglo...",,"0.074*patient, 0.018*treatment, 0.016*case, 0...."
7,8zchiykl,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.190059,"liver,sepsis,ARDS,respiratory distress syndrome",,"0.074*patient, 0.018*treatment, 0.016*case, 0...."
8,8qnrcgnk,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.026016,"bilirubin IXα,stress protein,Heme oxygenase-1...",,"0.074*patient, 0.018*treatment, 0.016*case, 0...."
9,jg13scgo,"0.035*study,0.019*datum,0.017*analysis,0.015*r...",0.374740,"—,outbreaks,complaint,RODS",,"0.035*study, 0.019*datum, 0.017*analysis, 0.01..."


In [36]:
topics = topics.assign(rec_id=np.arange(len(topics))).reset_index(drop=True)


In [37]:
topics['rec_id'] = 'T0_' + topics['rec_id'].astype(str)


In [38]:
csv_data = topics.to_csv(r'C:/Users/Iro Sfoungari/Desktop/Final_Topic_0_with_keywords.csv', index = False)

In [ ]:

0.023*virus,0.020*viral,

<h2> For keywords that became nodes!<h2>

In [39]:
topics.head(50)

,cord_uid,Topic,Probability,Entities,Intersection,Final_Topic_Keywords,rec_id
0,ug7v899j,"0.055*virus,0.017*human,0.017*viral,0.014*gene...",0.029789,"Patients,M. pneumoniae cultures,Jeddah,Mycopl...",,"0.055*virus, 0.017*human, 0.017*viral, 0.014*g...",T0_0
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573,phagocyte-derived heme proteins myeloperoxida...,0.014*lung,"0.074*patient, 0.018*treatment, 0.016*case, 0....",T0_1
2,ejv2xln0,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.056605,"metalloproteinase,collectin,SP-D,complex orga...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*...",T0_2
3,2b73a28n,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.112175,"ET-1,Endothelin-1,smooth muscle,airways,pulmo...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*...",T0_3
4,9785vg6d,"0.099*cell,0.024*infection,0.023*expression,0....",0.283934,"adhesion molecules,chemoattractant cytokines,...",,"0.099*cell, 0.024*infection, 0.023*expression,...",T0_4
5,zjufx4fo,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.347580,"subgenomic RNA molecules,TRS mutants,equine,Ni...",,"0.053*protein, 0.023*virus, 0.022*sars, 0.020*...",T0_5
6,5yhe786e,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.303051,"patient,patients,organ,organ failure,haemoglo...",,"0.074*patient, 0.018*treatment, 0.016*case, 0....",T0_6
7,8zchiykl,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.190059,"liver,sepsis,ARDS,respiratory distress syndrome",,"0.074*patient, 0.018*treatment, 0.016*case, 0....",T0_7
8,8qnrcgnk,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.026016,"bilirubin IXα,stress protein,Heme oxygenase-1...",,"0.074*patient, 0.018*treatment, 0.016*case, 0....",T0_8
9,jg13scgo,"0.035*study,0.019*datum,0.017*analysis,0.015*r...",0.374740,"—,outbreaks,complaint,RODS",,"0.035*study, 0.019*datum, 0.017*analysis, 0.01...",T0_9


In [46]:
len(topics)

269233

In [50]:
result1=topics[topics['Intersection']!='']

In [51]:
result1.head()

,cord_uid,Topic,Probability,Entities,Intersection,Final_Topic_Keywords,rec_id
1,02tnwd4m,"0.074*patient,0.018*treatment,0.016*case,0.015...",0.068573,phagocyte-derived heme proteins myeloperoxida...,0.014*lung,"0.074*patient, 0.018*treatment, 0.016*case, 0....",T0_1
10,5tkvsudh,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.236241,"ornithine decarboxylase,protein antizyme,fissi...",0.053*protein,"0.023*virus, 0.022*sars, 0.020*viral, 0.018*bi...",T0_10
11,6lvn10f4,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.545241,"cellular factors,hnRNP A1,250 kDa cellular pr...","0.020*viral,0.023*virus,0.053*protein","0.022*sars, 0.018*bind, 0.013*target, 0.012*ho...",T0_11
13,mcuixluu,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.416866,"microtubule,L4R,intracellular enveloped virus,...","0.020*viral,0.023*virus","0.053*protein, 0.022*sars, 0.018*bind, 0.013*t...",T0_13
23,e62cfqt7,"0.053*protein,0.023*virus,0.022*sars,0.020*vir...",0.075778,"amino terminus,green fluorescent protein,GFP,...",0.053*protein,"0.023*virus, 0.022*sars, 0.020*viral, 0.018*bi...",T0_23


In [52]:
len(result1)

46787

In [53]:
result1=result1.drop(['Topic', 'Probability', 'Entities', 'Final_Topic_Keywords'], axis = 1) 


In [54]:
result1.head(12)

,cord_uid,Intersection,rec_id
1,02tnwd4m,0.014*lung,T0_1
10,5tkvsudh,0.053*protein,T0_10
11,6lvn10f4,"0.020*viral,0.023*virus,0.053*protein",T0_11
13,mcuixluu,"0.020*viral,0.023*virus",T0_13
23,e62cfqt7,0.053*protein,T0_23
25,zowp10ts,0.024*infection,T0_25
27,snqdma0s,0.053*protein,T0_27
29,754nln40,"0.023*virus,0.053*protein",T0_29
34,cgl34ykt,0.023*virus,T0_34
35,ajlctjeb,0.020*viral,T0_35


In [55]:
splitted = result1['Intersection'].str.split(',\s*')
l = splitted.str.len()

df1 = pd.DataFrame({'rec_id': np.repeat(result1['rec_id'].values, l),
                     'Intersection':np.concatenate(splitted.values)}, columns=['rec_id','Intersection'])

In [56]:
df1.head(5)

,rec_id,Intersection
0,T0_1,0.014*lung
1,T0_10,0.053*protein
2,T0_11,0.020*viral
3,T0_11,0.023*virus
4,T0_11,0.053*protein


In [57]:
#df2= df1['Intersection'].str.split("\*", expand=True)
df2= df1['Intersection'].str.split("\*", expand=True)


In [58]:
df2.head()

,0,1
0,0.014,lung
1,0.053,protein
2,0.020,viral
3,0.023,virus
4,0.053,protein


In [59]:
df3=pd.concat([df1['rec_id'], df2], axis=1)


In [60]:
df3.head()

,rec_id,0,1
0,T0_1,0.014,lung
1,T0_10,0.053,protein
2,T0_11,0.020,viral
3,T0_11,0.023,virus
4,T0_11,0.053,protein


In [61]:
df3.columns = ['rec_id', 'Probability', 'Word']


In [62]:
len(df3)

64915

In [63]:
csv_data = result1.to_csv(r'C:/Users/Iro Sfoungari/Desktop/KeywordsBecameEntities_Topic_0.csv', index = False)